In [1]:
# !conda env create -f /kaggle/input/condaenv/environment.yml
# !conda init
# !conda activate pestvision
# !wget -P /kaggle/input/ https://storage.googleapis.com/npss-pestvision-data/pestvision_data.zip
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.7/40.7 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.9/777.9 kB 21.2 MB/s eta 0:00:00


In [2]:
import os
os.environ['WANDB_DISABLED'] = 'true'
import numpy as np
import torch
import yaml

In [3]:
root = '/kaggle/input/pestvisiondata/pestvision_data/'
dset_paths = ['synthetic_data/DeepImageBlendingData/run3/RiceLeafs/images/train',
             'synthetic_data/DeepImageBlendingData/run3/paddy-disease-classification/images/train',
             'synthetic_data/Libcom_HarmonizationData_PCTNet/run1/RiceLeafs/images/train',
             'synthetic_data/Libcom_HarmonizationData_PCTNet/run1/paddy-disease-classification/images/train']
dset_paths = [os.path.join(root,i) for i in dset_paths]
val_paths = ["/".join(i.split('/')[:-1]+['val']) for i in dset_paths]

In [4]:
# object detection code base with abstract class

import os
from abc import ABC, abstractmethod
from ultralytics import YOLO
import wandb
from wandb.integration.ultralytics import add_wandb_callback


class AbstractPestDetection(ABC):
    """
    Abstract class for pest detection
    """

    def __init__(self, device):
        self.device = device

    @abstractmethod
    def load_model(self, model_path):
        pass

    @abstractmethod
    def train(self):
        pass

    # TODO: implement evaluate method
    # @abstractmethod
    # def evaluate(self):
    #     pass


class PestDetection_yolov8(AbstractPestDetection):
    """
    YOLOv8 model for pest detection
    """

    def __init__(self, device):
        super().__init__(device)
        self.model = None

    def load_model(self, model_path):
        """
        Load the YOLOv8 model

        Parameters:

         model_path (str): path to the model checkpoint
        """

        self.model = YOLO(model_path)

    def train(self,**kwargs):

        results = self.model.train(**kwargs)
        self.model.val()

        return results

    # TODO: implment evaluate method
    # def evaluate(self):
    #     pass


wandb: WARNING This integration is tested and supported for ultralytics v8.0.238 and below.
wandb: WARNING             Please report any issues to https://github.com/wandb/wandb/issues with the tag `yolov8`.


In [5]:
def make_dsetconfig(train,val):

    d_config = {
        "train": train,
        "val": val,

        "nc": 102, # number of classes
        "names": [str(i) for i in range(1,103)] 
    }
    filename = 'data.yaml'
    with open(filename, 'w') as file:
        yaml.dump(d_config, file, default_flow_style=False)
    return filename

In [6]:
def train(checkpoint,train,val):
    if torch.cuda.is_available():
        device = torch.device('cuda')
        print("CUDA is available. Using GPU.")
    else:
        device = torch.device('cpu')
        print("CUDA is not available. Using CPU.")
    model = PestDetection_yolov8(device)
    model.load_model(checkpoint)
#     wandb_kwargs = {"project": None, "entity": None, "group": None, "job_type": None}
    results = model.train(data=make_dsetconfig(train,val), epochs=100, imgsz=512, save_period=10,batch=-1, plots=True)

In [7]:
train('yolov8n.pt', dset_paths, val_paths[1])

CUDA is available. Using GPU.


100%|██████████| 6.23M/6.23M [00:00<00:00, 106MB/s]


Ultralytics YOLOv8.2.21 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16276MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=data.yaml, epochs=100, time=None, patience=100, batch=-1, imgsz=512, save=True, save_period=10, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True

100%|██████████| 755k/755k [00:00<00:00, 23.0MB/s]
2024-05-25 12:34:29,912	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-05-25 12:34:31,381	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


Overriding model.yaml nc=80 with nc=102

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralyti

...

Ultralytics YOLOv8.2.21 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16276MiB)
Model summary (fused): 168 layers, 3341330 parameters, 0 gradients, 9.6 GFLOPs


val: Scanning /kaggle/input/pestvisiondata/pestvision_data/synthetic_data/DeepImageBlendingData/run3/paddy-disease-classification/labels/val... 2073 images, 233 backgrounds, 0 corrupt: 100%|██████████| 2306/2306 [00:03<00:00, 627.42it/s]


val: WARNING ⚠️ Cache directory /kaggle/input/pestvisiondata/pestvision_data/synthetic_data/DeepImageBlendingData/run3/paddy-disease-classification/labels is not writeable, cache not saved.


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 45/45 [00:23<00:00,  1.95it/s]


                   all       2306      11390       0.26      0.266      0.222      0.123
                     1       2306         77      0.362      0.519      0.405      0.189
                     2       2306         83      0.171     0.0964     0.0548     0.0359
                     3       2306         15          0          0     0.0343     0.0152
                     4       2306        110      0.385      0.409      0.285      0.136
                     5       2306         47      0.304      0.447      0.374      0.239
                     6       2306         62      0.251      0.108      0.122     0.0616
                     7       2306         21      0.319     0.0456     0.0717     0.0515
                     8       2306         74     0.0793     0.0541     0.0584     0.0279
                     9       2306         48      0.284      0.174      0.156     0.0747
                    10       2306         33      0.119     0.0303     0.0444     0.0262
                    1